# Spam or Ham

## Final Project - Intro to Deep Learning
## BGU Winter 2021

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
from google.colab import drive
#drive.mount('/content/gdrive/deep_learning/FinalProject/data')

path_route = os.walk('/content/gdrive/deep_learning/FinalProject/data')

hamData, spamData = [], []

for root, directory, file in path_route:
  if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                hamData.append(" ".join(ip.readlines()))
  elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                spamData.append(" ".join(ip.readlines()))

# remove all redundent data

allHamData = list(set(hamData))
allSpamData = list(set(spamData))

# storing it in a dataframe

hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})

# checking how it looks

raw_df.sample(5)